In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [2]:
import numpy as np
import pickle as pic

from sklearn.model_selection import train_test_split

In [3]:
import deep_learning

2024-06-12 09:16:51.843797: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-12 09:16:51.893151: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-12 09:16:52.834310: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
if True:
    with open("splitted_round_4.pic", "rb") as r:
        X_training, X_val, X_extraction = pic.load(r)
    with open("splitted_labels_1000000.pic", "rb") as r:
        y_training, y_val, y_extraction = pic.load(r)
else:
    with open("traces_round_4_adjusted_only.pic", "rb") as r:
        traces_round_4 = pic.load(r)
    with open("labels_1000000.pic", "rb") as r:
        rws_perms_labels, round_perms_labels, copy_perms_labels, rws_masks_labels, round_masks_labels = pic.load(r)

    X_total, y_total = deep_learning.prepare_data_dl(traces_round_4, round_perms_labels, copy_perms_labels, round_masks_labels, rws_perms_labels, rws_masks_labels)

    profile, test = train_test_split(np.arange(X_total.shape[0]), train_size=750_000, random_state=0)

    X_profiling, X_extraction = X_total[profile], X_total[test]
    train, val = train_test_split(np.arange(X_profiling.shape[0]), test_size=0.1, random_state=0)
    X_training, X_val = X_profiling[train], X_profiling[val]

    with open("splitted_round_4.pic", "wb") as w:
        pic.dump((X_training, X_val, X_extraction), w)

    if False:
        y_profiling = {}
        y_training = {}
        y_val = {}
        y_extraction = {}
        for label in y_total.keys():
            print(label, end="\r")
            y_profiling[label], y_extraction[label] = y_total[label][profile], y_total[label][test]
            y_training[label], y_val[label] = y_profiling[label][train], y_profiling[label][val]

        with open("splitted_labels_1000000.pic", "wb") as w:
            pic.dump((y_training, y_val, y_extraction), w)
    else:
        with open("splitted_labels_1000000.pic", "rb") as r:
            y_training, y_val, y_extraction = pic.load(r)


In [5]:
resnet = deep_learning.ResNetSCA(network="orig_round_4", epochs=1000, dataset_size=X_training.shape[0])
try:
    deep_learning.check_file_exists("./resnet_models/resnet_750000_orig_round_4_adjusted.keras")
    from tensorflow.keras.models import load_model
    resnet.model = load_model("./resnet_models/resnet_750000_orig_round_4_adjusted.keras")
except ValueError:
    pass

2024-06-12 09:16:55.855126: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-12 09:16:55.893995: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-12 09:16:55.894063: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-12 09:16:55.906962: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:1a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-12 09:16:55.907060: I external/local_xla/xla/stream_executor

In [6]:
train_gen = deep_learning.DataGenerator(X_training, y_training)
val_gen = deep_learning.DataGenerator(X_val, y_val)

In [7]:
history = resnet.train_model_generator(train_gen, val_gen, "./resnet_models/resnet_750000_orig_round_4_adjusted.keras", patience=10)

Epoch 1/1000


2024-06-12 09:17:17.477831: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
/root/Pierugo/protected_side_channel_attack/.venv_linux/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1718176637.687996   48896 service.cc:145] XLA service 0x7f3530076310 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1718176637.688051   48896 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-06-12 09:17:18.282041: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268

 6599/10547 ━━━━━━━━━━━━━━━━━━━━ 4:03 62ms/step - block_perm_4_output_accuracy: 0.1438 - loss: 41.4195 - mask_4_0_0_output_accuracy: 0.0624 - mask_4_0_1_output_accuracy: 0.0630 - mask_4_1_0_output_accuracy: 0.0622 - mask_4_1_1_output_accuracy: 0.0619 - mask_4_2_0_output_accuracy: 0.0624 - mask_4_2_1_output_accuracy: 0.0619 - mask_4_3_0_output_accuracy: 0.0629 - mask_4_3_1_output_accuracy: 0.0629 - mask_4_4_0_output_accuracy: 0.0623 - mask_4_4_1_output_accuracy: 0.0629 - mask_4_5_0_output_accuracy: 0.0621 - mask_4_5_1_output_accuracy: 0.0620 - mask_4_6_0_output_accuracy: 0.0626 - mask_4_6_1_output_accuracy: 0.0622

2024-06-12 09:24:28.231435: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 11.61GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
I0000 00:00:1718177080.588444   48896 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_41', 120 bytes spill stores, 120 bytes spill loads



10547/10547 ━━━━━━━━━━━━━━━━━━━━ 735s 66ms/step - block_perm_4_output_accuracy: 0.1437 - loss: 41.3849 - mask_4_0_0_output_accuracy: 0.0625 - mask_4_0_1_output_accuracy: 0.0627 - mask_4_1_0_output_accuracy: 0.0622 - mask_4_1_1_output_accuracy: 0.0621 - mask_4_2_0_output_accuracy: 0.0624 - mask_4_2_1_output_accuracy: 0.0620 - mask_4_3_0_output_accuracy: 0.0627 - mask_4_3_1_output_accuracy: 0.0631 - mask_4_4_0_output_accuracy: 0.0624 - mask_4_4_1_output_accuracy: 0.0629 - mask_4_5_0_output_accuracy: 0.0622 - mask_4_5_1_output_accuracy: 0.0621 - mask_4_6_0_output_accuracy: 0.0625 - mask_4_6_1_output_accuracy: 0.0625 - val_block_perm_4_output_accuracy: 0.1408 - val_loss: 58.2287 - val_mask_4_0_0_output_accuracy: 0.0615 - val_mask_4_0_1_output_accuracy: 0.0619 - val_mask_4_1_0_output_accuracy: 0.0649 - val_mask_4_1_1_output_accuracy: 0.0624 - val_mask_4_2_0_output_accuracy: 0.0610 - val_mask_4_2_1_output_accuracy: 0.0621 - val_mask_4_3_0_output_accuracy: 0.0622 - val_mask_4_3_1_output_accur